# LangChain & LangGraph Practice: Building Agentic AI Workflows

Welcome to this hands-on practice notebook for building Agentic AI workflows using LangChain and LangGraph!

## What You'll Learn:
- 🔧 Set up LangChain and LangGraph environments
- 🤖 Create simple and complex AI agents
- 🔗 Build graph-based workflows with multiple agents
- 🛠️ Integrate external tools and APIs
- 💾 Implement memory and state management
- 🔀 Create conditional workflow paths
- 🧪 Test and debug agent workflows

## Prerequisites:
- Basic Python knowledge
- OpenAI API key (or other LLM provider)
- Understanding of AI/ML concepts (helpful but not required)

Let's start building!

## 1. Setup and Install Dependencies

First, let's install all the required packages for our LangChain and LangGraph practice.

In [1]:
# Install required packages
# Run this cell if packages are not already installed

import subprocess
import sys

packages = [
    "langchain",
    "langchain-community", 
    "langchain-openai",
    "langgraph",
    "python-dotenv",
    "matplotlib",
    "pandas"
]

for package in packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"✅ {package} is already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

✅ langchain is already installed
✅ langchain-community is already installed
✅ langchain-openai is already installed
✅ langgraph is already installed
📦 Installing python-dotenv...



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


✅ matplotlib is already installed
✅ pandas is already installed


## 2. Import Required Libraries

Import all the necessary libraries for building our agentic workflows.

In [3]:
# Core imports
import os
import json
from typing import TypedDict, Annotated, Literal, List, Dict, Any
from dotenv import load_dotenv

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain import hub

# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
# Note: ToolExecutor has been moved/deprecated in newer versions
# We'll create tools directly when needed

# Utility imports
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

print("📚 All libraries imported successfully!")
print("🚀 Ready to build agentic workflows!")
print("ℹ️ Note: ToolExecutor import updated for compatibility")

Matplotlib is building the font cache; this may take a moment.


📚 All libraries imported successfully!
🚀 Ready to build agentic workflows!
ℹ️ Note: ToolExecutor import updated for compatibility


## 3. Configure API Keys and Environment

⚠️ **Important**: You'll need to set up your API keys to run the examples.

### Required: LLM Provider API Key
You need an API key from an LLM provider like OpenAI, Anthropic, etc.

### Option 1: Create a `.env` file
Create a `.env` file in your project directory with:
```
OPENAI_API_KEY=your_openai_api_key_here
```

### Option 2: Set directly in the notebook (less secure)
Uncomment and modify the lines below.

### Optional: LangSmith (Monitoring & Debugging)
LangSmith is LangChain's optional monitoring service. It's separate from LangChain itself:
```
LANGCHAIN_TRACING_V2=true
LANGCHAIN_API_KEY=your_langsmith_api_key_here
```

**Note**: LangChain framework itself is free and open-source - no API key required!

In [ ]:
# Load environment variables
load_dotenv()

# Option 2: Set API keys directly (uncomment and modify)
# os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = "your_langsmith_api_key_here"

# Check if LLM API key is set (REQUIRED)
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("✅ OpenAI API key is configured")
    print(f"🔑 Key starts with: {api_key[:8]}...")
else:
    print("❌ OpenAI API key not found")
    print("📝 Please set OPENAI_API_KEY in your .env file or directly in the cell above")
    print("💡 You can get an API key from: https://platform.openai.com/api-keys")

# Optional: Check LangSmith setup (for monitoring/debugging)
if os.getenv("LANGCHAIN_API_KEY"):
    print("✅ LangSmith tracing is configured (optional)")
    print("🔍 This enables monitoring and debugging of your chains")
else:
    print("ℹ️ LangSmith tracing not configured (optional)")
    print("💡 LangSmith is a separate monitoring service - not required for learning!")

print("\n🎯 Summary:")
print("• LangChain = Free, open-source framework (no API key)")
print("• OpenAI = LLM provider (API key required)")  
print("• LangSmith = Optional monitoring service (separate API key)")

❌ OpenAI API key not found
📝 Please set OPENAI_API_KEY in your .env file or directly in the cell above
ℹ️ LangSmith tracing not configured (optional for learning)


## 4. Create a Simple LangChain Agent

Let's start with a basic LangChain agent that can use tools to perform tasks.

### Exercise 4.1: Simple Calculator Agent
We'll create an agent that can perform mathematical calculations.

In [ ]:
# Define a simple calculator tool
def calculator_tool(expression: str) -> str:
    """
    Performs mathematical calculations.
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 3 * 4")
    Returns:
        The result of the calculation
    """
    try:
        # Be careful with eval in production! This is for learning purposes.
        result = eval(expression)
        return f"The result of {expression} is: {result}"
    except Exception as e:
        return f"Error calculating {expression}: {str(e)}"

# Create the tool
calculator = Tool(
    name="calculator",
    description="Useful for performing mathematical calculations. Input should be a mathematical expression.",
    func=calculator_tool
)

# Test the tool
print("🧮 Testing calculator tool:")
print(calculator.run("15 + 25"))
print(calculator.run("(10 * 5) / 2"))

In [ ]:
# Create the LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

# Create tools list
tools = [calculator]

# Get the prompt from LangChain hub
try:
    prompt = hub.pull("hwchase17/openai-functions-agent")
    print("✅ Prompt loaded from LangChain hub")
except:
    # Fallback prompt if hub is not available
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that can use tools to help answer questions."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    print("✅ Using fallback prompt")

# Create the agent
agent = create_openai_functions_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=3
)

print("🤖 Calculator agent created successfully!")

In [ ]:
# Test the agent with different mathematical queries
test_queries = [
    "What is 15 multiplied by 23?",
    "Calculate the area of a circle with radius 5 (use π = 3.14159)",
    "If I invest $1000 at 5% annual interest for 3 years, how much will I have? Use compound interest formula."
]

print("🧪 Testing the calculator agent:\n")

for i, query in enumerate(test_queries, 1):
    print(f"📝 Test {i}: {query}")
    try:
        result = agent_executor.invoke({"input": query})
        print(f"✅ Result: {result['output']}")
    except Exception as e:
        print(f"❌ Error: {str(e)}")
    print("-" * 60)

## 5. Build Basic LangGraph Workflow

Now let's create a simple graph-based workflow using LangGraph. This workflow will analyze a problem and provide recommendations.

### Exercise 5.1: Problem Analysis Workflow

In [ ]:
# Define the state for our workflow
class AnalysisState(TypedDict):
    messages: Annotated[list, add_messages]
    problem: str
    analysis: str
    recommendations: str
    current_step: str

# Define workflow nodes
def analyze_problem(state: AnalysisState) -> AnalysisState:
    """Analyze the user's problem"""
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert analyst. Analyze the given problem thoroughly, identifying key issues, root causes, and important factors."),
        ("human", "Problem: {problem}\n\nProvide a detailed analysis:")
    ])
    
    chain = prompt | llm | StrOutputParser()
    analysis = chain.invoke({"problem": state["problem"]})
    
    return {
        **state,
        "analysis": analysis,
        "current_step": "analysis_complete"
    }

def generate_recommendations(state: AnalysisState) -> AnalysisState:
    """Generate recommendations based on the analysis"""
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Based on the analysis provided, generate 3-5 practical, actionable recommendations with clear steps."),
        ("human", "Problem: {problem}\n\nAnalysis: {analysis}\n\nProvide specific recommendations:")
    ])
    
    chain = prompt | llm | StrOutputParser()
    recommendations = chain.invoke({
        "problem": state["problem"],
        "analysis": state["analysis"]
    })
    
    return {
        **state,
        "recommendations": recommendations,
        "current_step": "recommendations_complete"
    }

print("✅ Workflow nodes defined!")

In [ ]:
# Create the workflow graph
workflow = StateGraph(AnalysisState)

# Add nodes to the graph
workflow.add_node("analyze", analyze_problem)
workflow.add_node("recommend", generate_recommendations)

# Define the flow
workflow.set_entry_point("analyze")
workflow.add_edge("analyze", "recommend")
workflow.add_edge("recommend", END)

# Compile the graph
app = workflow.compile()

print("🔄 LangGraph workflow created!")

# Test the workflow
test_problems = [
    "I want to learn machine learning but don't know where to start and feel overwhelmed by all the options.",
    "My team is struggling with communication in remote work setup.",
    "I need to decide between Python and JavaScript for my next web development project."
]

print("\n🧪 Testing the analysis workflow:\n")

for i, problem in enumerate(test_problems, 1):
    print(f"📝 Problem {i}: {problem}")
    
    result = app.invoke({
        "messages": [],
        "problem": problem,
        "analysis": "",
        "recommendations": "",
        "current_step": "start"
    })
    
    print(f"\n📊 Analysis:\n{result['analysis']}")
    print(f"\n💡 Recommendations:\n{result['recommendations']}")
    print("-" * 80)

## 6. Create Multi-Agent Workflow

Let's build a more complex workflow with multiple specialized agents that collaborate on tasks.

### Exercise 6.1: Content Creation Workflow
We'll create a workflow with different agents for different types of content creation.

In [ ]:
# Define state for multi-agent workflow
class MultiAgentState(TypedDict):
    messages: Annotated[list, add_messages]
    task_type: str
    user_input: str
    content: str
    review_feedback: str
    final_output: str
    iteration_count: int

# Classifier agent
def classify_task(state: MultiAgentState) -> MultiAgentState:
    """Classify the type of content creation task"""
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Classify the user's request into one of these categories:
        - 'technical': Code, documentation, technical explanations
        - 'creative': Stories, poems, creative writing
        - 'business': Reports, proposals, business content
        - 'educational': Tutorials, explanations, learning materials
        
        Respond with just the category name."""),
        ("human", "{input}")
    ])
    
    chain = prompt | llm | StrOutputParser()
    task_type = chain.invoke({"input": state["user_input"]}).strip().lower()
    
    return {**state, "task_type": task_type, "iteration_count": 0}

# Specialized agents
def technical_agent(state: MultiAgentState) -> MultiAgentState:
    """Create technical content"""
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a technical expert. Create accurate, detailed technical content with examples and clear explanations."),
        ("human", "{input}")
    ])
    
    chain = prompt | llm | StrOutputParser()
    content = chain.invoke({"input": state["user_input"]})
    
    return {**state, "content": content}

def creative_agent(state: MultiAgentState) -> MultiAgentState:
    """Create creative content"""
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.8)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a creative writer. Create engaging, imaginative content with vivid descriptions and compelling narratives."),
        ("human", "{input}")
    ])
    
    chain = prompt | llm | StrOutputParser()
    content = chain.invoke({"input": state["user_input"]})
    
    return {**state, "content": content}

def business_agent(state: MultiAgentState) -> MultiAgentState:
    """Create business content"""
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a business professional. Create clear, professional business content with structured formatting and actionable insights."),
        ("human", "{input}")
    ])
    
    chain = prompt | llm | StrOutputParser()
    content = chain.invoke({"input": state["user_input"]})
    
    return {**state, "content": content}

def educational_agent(state: MultiAgentState) -> MultiAgentState:
    """Create educational content"""
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.4)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an educator. Create clear, step-by-step educational content with examples and practical exercises."),
        ("human", "{input}")
    ])
    
    chain = prompt | llm | StrOutputParser()
    content = chain.invoke({"input": state["user_input"]})
    
    return {**state, "content": content}

print("✅ Multi-agent system defined!")

In [ ]:
# Routing logic
def route_to_agent(state: MultiAgentState) -> Literal["technical", "creative", "business", "educational"]:
    """Route to the appropriate agent based on task type"""
    task_type = state["task_type"]
    return task_type if task_type in ["technical", "creative", "business", "educational"] else "educational"

# Review agent
def review_content(state: MultiAgentState) -> MultiAgentState:
    """Review the generated content"""
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Review the content and provide feedback. Rate it 1-10 and suggest improvements.
        If the score is 8 or above, respond with 'APPROVED: [score]/10 - [brief positive feedback]'
        If below 8, provide specific improvement suggestions."""),
        ("human", "Content to review: {content}")
    ])
    
    chain = prompt | llm | StrOutputParser()
    feedback = chain.invoke({"content": state["content"]})
    
    return {
        **state, 
        "review_feedback": feedback,
        "iteration_count": state["iteration_count"] + 1
    }

def should_finalize(state: MultiAgentState) -> Literal["finalize", "revise"]:
    """Decide whether to finalize or revise"""
    feedback = state["review_feedback"].lower()
    max_iterations = 2
    
    if "approved" in feedback or state["iteration_count"] >= max_iterations:
        return "finalize"
    else:
        return "revise"

def finalize_output(state: MultiAgentState) -> MultiAgentState:
    """Finalize the output"""
    return {**state, "final_output": state["content"]}

def revise_content(state: MultiAgentState) -> MultiAgentState:
    """Revise content based on feedback"""
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Revise the content based on the feedback provided. Improve quality and address concerns."),
        ("human", "Original request: {user_input}\n\nCurrent content: {content}\n\nFeedback: {feedback}\n\nProvide improved version:")
    ])
    
    chain = prompt | llm | StrOutputParser()
    revised_content = chain.invoke({
        "user_input": state["user_input"],
        "content": state["content"],
        "feedback": state["review_feedback"]
    })
    
    return {**state, "content": revised_content}

print("✅ Routing and review logic defined!")

In [ ]:
# Create the multi-agent workflow
multi_workflow = StateGraph(MultiAgentState)

# Add all nodes
multi_workflow.add_node("classify", classify_task)
multi_workflow.add_node("technical", technical_agent)
multi_workflow.add_node("creative", creative_agent)
multi_workflow.add_node("business", business_agent)
multi_workflow.add_node("educational", educational_agent)
multi_workflow.add_node("review", review_content)
multi_workflow.add_node("revise", revise_content)
multi_workflow.add_node("finalize", finalize_output)

# Define the flow
multi_workflow.set_entry_point("classify")

# Route to appropriate agent
multi_workflow.add_conditional_edges(
    "classify",
    route_to_agent,
    {
        "technical": "technical",
        "creative": "creative", 
        "business": "business",
        "educational": "educational"
    }
)

# All agents go to review
multi_workflow.add_edge("technical", "review")
multi_workflow.add_edge("creative", "review")
multi_workflow.add_edge("business", "review")
multi_workflow.add_edge("educational", "review")

# Conditional routing after review
multi_workflow.add_conditional_edges(
    "review",
    should_finalize,
    {
        "finalize": "finalize",
        "revise": "revise"
    }
)

# After revision, go back to review
multi_workflow.add_edge("revise", "review")
multi_workflow.add_edge("finalize", END)

# Compile the multi-agent workflow
multi_app = multi_workflow.compile()

print("🤖 Multi-agent workflow created!")

In [ ]:
# Test the multi-agent workflow
test_requests = [
    "Write a Python function to implement bubble sort with comments",
    "Create a short story about a time traveler who gets stuck in 1920s Paris",
    "Draft a business proposal for a new employee wellness program",
    "Explain how photosynthesis works with simple examples for high school students"
]

print("🧪 Testing multi-agent workflow:\n")

for i, request in enumerate(test_requests, 1):
    print(f"📝 Request {i}: {request}")
    
    result = multi_app.invoke({
        "messages": [],
        "task_type": "",
        "user_input": request,
        "content": "",
        "review_feedback": "",
        "final_output": "",
        "iteration_count": 0
    })
    
    print(f"🏷️ Task Type: {result['task_type']}")
    print(f"🔄 Iterations: {result['iteration_count']}")
    print(f"📊 Final Review: {result['review_feedback'][:100]}...")
    print(f"✅ Final Output:\n{result['final_output'][:200]}...")
    print("-" * 80)

## 7. Implement Tool Integration

Let's add external tools and APIs to our agents for enhanced capabilities.

## 8. Add Memory and State Management

Implement conversation memory and state persistence.

## 9. Create Conditional Workflow Paths

Build dynamic workflows with conditional branching.

## 10. Test and Debug Agent Workflows

Create comprehensive test cases and debugging strategies.

### 🎯 Practice Exercises

Try these exercises to deepen your understanding:

1. **Custom Tool Creation**: Create a tool that fetches weather data or current time
2. **Memory Enhancement**: Add long-term memory to store user preferences
3. **Complex Routing**: Create a workflow that routes based on multiple conditions
4. **Error Handling**: Add robust error handling and recovery mechanisms
5. **Performance Optimization**: Implement caching and optimize LLM calls

### 📚 Next Steps

- Explore LangSmith for monitoring and debugging
- Try different LLM providers (Anthropic, Google, etc.)
- Build domain-specific agents for your use case
- Integrate with external APIs and databases
- Deploy your agents to production

### 🔗 Useful Resources

- [LangChain Documentation](https://python.langchain.com/docs/get_started/introduction)
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [LangSmith for Monitoring](https://smith.langchain.com/)
- [OpenAI API Reference](https://platform.openai.com/docs/api-reference)

Happy building! 🚀

## 🚀 Quick Demo: Understanding LangChain Basics

Let's start with a simple demonstration to understand how LangChain works, even without an API key!

In [1]:
# Quick Demo: LangChain Prompt Templates (No API Key Required)
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

print("🔗 Understanding LangChain Building Blocks")
print("=" * 60)

# 1. Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that explains {subject} concepts clearly."),
    ("human", "Explain {topic} in simple terms with a practical example.")
])

print("✅ Step 1: Prompt Template Created")
print("📝 This template has two variables: {subject} and {topic}")
print()

# 2. Show how prompt formatting works
subjects_and_topics = [
    ("programming", "variables"),
    ("cooking", "seasoning"),
    ("gardening", "composting")
]

print("📋 See how the template gets filled with actual values:")
print()

for subject, topic in subjects_and_topics:
    # Format the prompt with actual values
    formatted_messages = prompt.format_messages(subject=subject, topic=topic)
    
    print(f"🎯 Subject: {subject}, Topic: {topic}")
    print(f"   System: {formatted_messages[0].content}")
    print(f"   Human:  {formatted_messages[1].content}")
    print()

# 3. Show the chain concept
print("🔗 The Chain Concept:")
print("   prompt | llm | output_parser")
print("   ↓")
print("   Input → Format Prompt → Send to LLM → Parse Response → Final Output")
print()

# 4. Show what the output parser does
output_parser = StrOutputParser()
print("🔧 Output Parser:")
print("   Converts LLM response objects into simple strings")
print("   Makes the output easy to work with in your code")

print("\n" + "=" * 60)
print("💡 Key Concepts Demonstrated:")
print("• Prompt templates with variables {like_this}")
print("• Message types: system (instructions) and human (user input)")
print("• The pipe operator | chains components together") 
print("• Output parsers clean up the LLM response")
print("\nNext: Try running this with an actual LLM!")

🔗 Understanding LangChain Building Blocks
✅ Step 1: Prompt Template Created
📝 This template has two variables: {subject} and {topic}

📋 See how the template gets filled with actual values:

🎯 Subject: programming, Topic: variables
   System: You are a helpful assistant that explains programming concepts clearly.
   Human:  Explain variables in simple terms with a practical example.

🎯 Subject: cooking, Topic: seasoning
   System: You are a helpful assistant that explains cooking concepts clearly.
   Human:  Explain seasoning in simple terms with a practical example.

🎯 Subject: gardening, Topic: composting
   System: You are a helpful assistant that explains gardening concepts clearly.
   Human:  Explain composting in simple terms with a practical example.

🔗 The Chain Concept:
   prompt | llm | output_parser
   ↓
   Input → Format Prompt → Send to LLM → Parse Response → Final Output

🔧 Output Parser:
   Converts LLM response objects into simple strings
   Makes the output easy to work

In [2]:
# Complete Example: What happens when you add an LLM to the chain
print("🤖 Complete LangChain Flow (Simulated)")
print("=" * 50)

# Simulate what would happen with a real LLM
class MockLLM:
    """A mock LLM that simulates responses for demonstration"""
    
    def invoke(self, messages):
        # Get the human message (the actual question)
        human_msg = messages[-1].content
        
        # Simple mock responses based on keywords
        if "variables" in human_msg.lower():
            return "Variables are like containers that store information in programming. For example, if you write 'name = \"Alice\"', you're storing the text 'Alice' in a container called 'name' that you can use later in your code."
        elif "seasoning" in human_msg.lower():
            return "Seasoning is adding salt, herbs, and spices to enhance food flavor. For example, adding a pinch of salt and some black pepper to scrambled eggs makes them taste much better than plain eggs."
        elif "composting" in human_msg.lower():
            return "Composting is recycling organic waste into nutrient-rich soil. For example, you can put fruit peels, coffee grounds, and fallen leaves in a bin, and over time they decompose into rich fertilizer for your garden."
        else:
            return "I'd be happy to explain that topic with a practical example!"

# Create the complete chain
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that explains {subject} concepts clearly."),
    ("human", "Explain {topic} in simple terms with a practical example.")
])

mock_llm = MockLLM()
output_parser = StrOutputParser()

print("🔗 Building the chain: prompt | mock_llm | output_parser")
print()

# Test the complete flow
test_cases = [
    ("programming", "variables"),
    ("cooking", "seasoning"), 
    ("gardening", "composting")
]

for subject, topic in test_cases:
    print(f"📝 Question: Explain {topic} in {subject}")
    
    # Step 1: Format the prompt
    formatted_messages = prompt.format_messages(subject=subject, topic=topic)
    print(f"   → Formatted prompt created")
    
    # Step 2: Send to LLM
    llm_response = mock_llm.invoke(formatted_messages)
    print(f"   → LLM generated response")
    
    # Step 3: Parse output
    final_output = output_parser.invoke(llm_response)
    print(f"   → Output parsed")
    
    print(f"✅ Final Answer: {final_output}")
    print("-" * 50)

print("\n💡 This is exactly what happens in a real LangChain application:")
print("1. Your input fills the prompt template variables")
print("2. The formatted prompt goes to the LLM (like GPT-3.5)")  
print("3. The LLM generates a response")
print("4. The output parser cleans it up for your app")
print("\n🔑 To make this work with real AI, just add your OpenAI API key!")

🤖 Complete LangChain Flow (Simulated)
🔗 Building the chain: prompt | mock_llm | output_parser

📝 Question: Explain variables in programming
   → Formatted prompt created
   → LLM generated response
   → Output parsed
✅ Final Answer: Variables are like containers that store information in programming. For example, if you write 'name = "Alice"', you're storing the text 'Alice' in a container called 'name' that you can use later in your code.
--------------------------------------------------
📝 Question: Explain seasoning in cooking
   → Formatted prompt created
   → LLM generated response
   → Output parsed
✅ Final Answer: Seasoning is adding salt, herbs, and spices to enhance food flavor. For example, adding a pinch of salt and some black pepper to scrambled eggs makes them taste much better than plain eggs.
--------------------------------------------------
📝 Question: Explain composting in gardening
   → Formatted prompt created
   → LLM generated response
   → Output parsed
✅ Final A